In [1]:
import sys
sys.path.insert(0, '../../')
import ccal
%matplotlib inline
%config InlineBackend.figure_formats = {'svg',}

import numpy as np
import pandas as pd

==================== Computational Cancer Analysis Library ====================
<15:26:23> Checking dependencies ...
<15:26:23> Using the following packages:
<15:26:23> 	matplotlib (v1.5.1)
<15:26:23> 	numpy (v1.10.4)
<15:26:23> 	pandas (v0.18.0)
<15:26:23> 	rpy2 (v2.7.9)
<15:26:23> 	scikit-learn (v0.17.1)
<15:26:23> 	scipy (v0.17.0)
<15:26:23> 	seaborn (v0.7.0)


# Get states

In [2]:
h = ccal.support.read_gct('../../../hematopoietic_cancer/result/ccle_hema/k9/ccle_hema_k_9_H.gct')
n_states_to_explore = [9]
nclustering = 3
states_filename = '~/Desktop/n_state_x_samples.txt'

In [ ]:
labels, cophenetic_correlation = ccal.analyze.get_states_from_h(h, n_states_to_explore, nclustering=nclustering, filename=states_filename)

<15:26:09> Comparing A3KAW_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE (1/166) vs. ...
<15:26:10> Comparing A4FUK_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE (2/166) vs. ...
<15:26:10> Comparing ALLSIL_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE (3/166) vs. ...
<15:26:11> Comparing AML193_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE (4/166) vs. ...
<15:26:11> Comparing AMO1_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE (5/166) vs. ...
<15:26:12> Comparing BCP1_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE (6/166) vs. ...
<15:26:12> Comparing BDCM_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE (7/166) vs. ...
<15:26:13> Comparing BL41_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE (8/166) vs. ...
<15:26:13> Comparing BL70_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE (9/166) vs. ...
<15:26:14> Comparing BV173_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE (10/166) vs. ...
<15:26:14> Comparing CA46_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE (11/166) vs. ...
<15:26:15> Comparing CI1_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE (12/166) vs. ...


# Map Onco GPS

In [3]:
n_state = 9
figure_filename='/Users/Kwat/Desktop/onco_gps.pdf'

labels = pd.read_csv(states_filename, index_col=0, sep='\t')
states = labels.ix[n_state, :-1].astype(int).tolist()

In [4]:
annotations = np.random.random_sample(len(states))

In [5]:
ccal.visualize.plot_onco_gps(h, n_state, states, output_filename=figure_filename)

TypeError: 'NoneType' object is not iterable